In [1]:
import pandas as pd
import cv2
from torch.utils.data import Dataset, DataLoader
from pycocotools.coco import COCO
import os
import numpy as np
from PIL import Image

import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch
from mmcv import Config
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor
from mmseg.datasets import (build_dataloader, build_dataset)
import mmcv

import os.path as osp
import numpy as np
from PIL import Image
import os
import json
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel


In [2]:
'''
경로 설정
'''

epoch = 'deeplabv3lplus_2_31' # csv 파일 이름
# model과 checkpoint 설정
cfg = Config.fromfile('/opt/ml/mmsegmentation/_MyModel/_models_/deeplabv3plus_r101b-d8_512x1024_80k_cityscapes.py')
checkpoint_path = '/opt/ml/mmsegmentation/_MyModel/work_dirs/deeplabv3plus_r101_2/best_mIoU_epoch_31.pth'



'''
아래부분은 건들지 않아도 됩니다.
'''

work_dir = '/opt/ml/mmsegmentation/_MyModel/submission'
root='/opt/ml/input/mmseg/images'

cfg.data.val.img_dir = root
cfg.data.val.test_mode = True
cfg.data.val.img_dir = 'images'
cfg.data.val.split = 'splits/valid.txt'
cfg.data.samples_per_gpu = 1
cfg.work_dir = work_dir
# cfg.model.decode_head.num_classes = 11
# cfg.model.auxiliary_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [3]:
dataset = build_dataset(cfg.data.val)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

2022-05-03 14:04:49,136 - mmseg - INFO - Loaded 654 images


In [4]:
# checkpoint path


model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

/opt/conda/lib/python3.8/site-packages/mmseg/models/backbones/resnet.py:431: UserWarning: DeprecationWarning: pretrained is a deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is a deprecated, '
/opt/conda/lib/python3.8/site-packages/mmseg/models/losses/cross_entropy_loss.py:225: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: /opt/ml/mmsegmentation/_MyModel/work_dirs/deeplabv3plus_r101_2/best_mIoU_epoch_31.pth


In [5]:
output = single_gpu_test(model, data_loader)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 654/654, 14.6 task/s, elapsed: 45s, ETA:     0s

In [7]:
input_size = 512
output_size = 256

submission = pd.read_csv("/opt/ml/mmsegmentation/_MyModel/submission/sample_submission.csv", index_col=None)
json_dir = os.path.join("/opt/ml/input/data/cv_val_1.json")

check_dir = '/opt/ml/input/mmseg/valid_check'
os.makedirs(check_dir,exist_ok=True)
os.makedirs(check_dir+'/csv',exist_ok=True)
os.makedirs(check_dir+'/image',exist_ok=True)


with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)
import matplotlib.pyplot as plt
from PIL import Image
# PredictionString 대입
palette = [[0,0,0],[192,0,128],[0,128,192],[0,128,64],[128,0,0],[64,0,128],[64,0,192],[192,128,64],[192,192,128],[64,64,128],[128,0,192]]

for image_id, predict in enumerate(output):

    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]
    
    temp_mask = []
    predict = predict.reshape(1, 512, 512)

    mask = predict
    temp_mask.append(mask)
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], input_size*input_size]).astype(int)
    string = oms.flatten()

    submission = pd.concat([submission, pd.DataFrame([{"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}])]
                                   , ignore_index=True)
                                   


submission.to_csv(os.path.join(check_dir, f'csv/submission_{epoch}.csv'), index=False)

## image 생성

In [4]:
dataset_path  = '/opt/ml/input/data'
path=os.path.join(check_dir, f'csv/submission_{epoch}.csv') # csv파일 경로. 필요에 따라 수정하시면 됩니다.
test_path = dataset_path + '/cv_val_1.json'
batch_size=16
result=pd.read_csv(path)
result.head()

,image_id,PredictionString
0,batch_01_vt/0003.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,batch_01_vt/0016.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,batch_01_vt/0018.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,batch_01_vt/0019.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,batch_01_vt/0022.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [8]:
category_names=['Backgroud',
 'General trash',
 'Paper',
 'Paper pack',
 'Metal',
 'Glass',
 'Plastic',
 'Styrofoam',
 'Plastic bag',
 'Battery',
 'Clothing']

palette = [[0,0,0],[192,0,128],[0,128,192],[0,128,64],[128,0,0],[64,0,128],[64,0,192],[192,128,64],[192,192,128],[64,64,128],[128,0,192]]

def get_classname(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id']==classID:
            return cats[i]['name']
    return "None"

class CustomDataLoader(Dataset):
    """COCO format"""
    def __init__(self, data_dir, mode = 'train', transform = None):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        
    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        # image_id = self.coco.getImgIds(imgIds=index)
        image_id = self.coco.getImgIds()[index]
        image_infos = self.coco.loadImgs(image_id)[0]
        
        # cv2 를 활용하여 image 불러오기
        images = cv2.imread(os.path.join(dataset_path, image_infos['file_name']))
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
        images /= 255.0
        
        if (self.mode in ('train', 'val')):
            ann_ids = self.coco.getAnnIds(imgIds=image_infos['id'])
            anns = self.coco.loadAnns(ann_ids)

            # Load the categories in a variable
            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)

            # masks : size가 (height x width)인 2D
            # 각각의 pixel 값에는 "category id" 할당
            # Background = 0
            masks = np.zeros((image_infos["height"], image_infos["width"]))
            # General trash = 1, ... , Cigarette = 10
            anns = sorted(anns, key=lambda idx : idx['area'], reverse=True)
            for i in range(len(anns)):
                className = get_classname(anns[i]['category_id'], cats)
                pixel_value = category_names.index(className)
                masks[self.coco.annToMask(anns[i]) == 1] = pixel_value
            masks = masks.astype(np.int8)
                        
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images, mask=masks)
                images = transformed["image"]
                masks = transformed["mask"]
            return images, masks, image_infos
        
        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
            return images, image_infos
    
    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

In [9]:
def collate_fn(batch):
    return tuple(zip(*batch))


test_transform = A.Compose([
                           ToTensorV2()
                           ])
test_dataset = CustomDataLoader(data_dir=test_path, mode='val', transform=test_transform)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          num_workers=4,
                                          collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.65s)
creating index...
index created!


In [10]:
# images 제거

import os.path as osp
from glob import glob
data_dir='/opt/ml/input/mmseg/valid_check/image'
image_paths = {x for x in glob(osp.join(data_dir, '*'))}

for image_path in image_paths:
    os.remove(image_path)

In [32]:
# images 생성

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
for i,(imgs, masks, image_infos) in enumerate(test_loader):
    temp_images = imgs
    temp_anns = image_infos
    temp_masks = masks
    for j in range(len(temp_images)):

        x=torch.Tensor(temp_images[j]).permute([1,2,0])*255
        x=x.numpy().astype(np.uint8)
        z=temp_masks[j].numpy()
        z=list(z.flatten())
        for k in range(len(z)):
            z[k]=palette[int(z[k])]
        z=torch.Tensor(z).reshape(512,512,3).numpy()

        y=result['PredictionString'][i*batch_size+j].split(" ")
        for k in range(len(y)):
            y[k]=palette[int(y[k])]
        y=torch.Tensor(y).reshape(512,512,3).numpy()

        temp_result=np.concatenate((x,z,y),1)
        temp_result=Image.fromarray(temp_result.astype(np.uint8))
        temp_result.save(os.path.join(data_dir,f"{int(temp_anns[j]['id']):04}.jpg"))


d
